In [1]:
import akshare as ak
import pandas as pd
import sys
sys.path.append('/Users/afirez/studio/python/auto-trade/examples/py_quant/common')
sys.path.append('/Users/afirez/studio/python/auto-trade/examples/py_quant/common/common_pg.py')

print("sys.path:",sys.path)
import common_pg as common_db

print(ak.__version__)

sys.path: ['/Users/afirez/studio/python/auto-trade/examples/py_quant/datasource/akshare/test', '/usr/local/anaconda3/envs/py38/lib/python38.zip', '/usr/local/anaconda3/envs/py38/lib/python3.8', '/usr/local/anaconda3/envs/py38/lib/python3.8/lib-dynload', '', '/usr/local/anaconda3/envs/py38/lib/python3.8/site-packages', '/usr/local/anaconda3/envs/py38/lib/python3.8/site-packages/IPython/extensions', '/Users/afirez/.ipython', '/Users/afirez/.local/lib/python3.8/site-packages', '/Users/afirez/studio/python/auto-trade/examples/py_quant/common', '/Users/afirez/studio/python/auto-trade/examples/py_quant/common/common_pg.py']
pg_host : 192.168.31.110 ,pg_user : postgres ,pg_db : k_house
1.6.64


In [2]:
# 实时行情数据
# 接口: stock_zh_a_spot
# 目标地址: http://vip.stock.finance.sina.com.cn/mkt/#hs_a
# 描述: A 股数据是从新浪财经获取的数据, 重复运行本函数会被新浪暂时封 IP, 建议增加时间间隔
# 限量: 单次返回所有 A 股上市公司的实时行情数据

stock_zh_a_spot_df = ak.stock_zh_a_spot()
print(stock_zh_a_spot_df)

            代码    名称    最新价   涨跌额    涨跌幅     买入     卖出     昨收     今开     最高  \
0     bj430047  诺思兰德  10.06 -0.10 -0.984  10.05  10.06  10.16  10.29  10.29   
1     bj430090  同辉信息   3.50 -0.03 -0.850   3.50   3.53   3.53   3.58   3.58   
2     bj430198  微创光电   5.32  0.00  0.000   5.31   5.32   5.32   5.30   5.32   
3     bj430418  苏轴股份   8.35  0.01  0.120   8.33   8.35   8.34   8.35   8.45   
4     bj430489  佳先股份   6.08 -0.02 -0.328   6.03   6.08   6.10   6.07   6.09   
...        ...   ...    ...   ...    ...    ...    ...    ...    ...    ...   
4830  sz301288  清研环境  26.00  1.57  6.427  26.00  26.01  24.43  24.46  26.38   
4831  sz301289  国缆检测  53.06  0.07  0.132  53.06  53.08  52.99  55.50  57.24   
4832  sz301298  东利机械  24.42  0.51  2.133  24.41  24.42  23.91  24.48  25.89   
4833  sz301302  华如科技  65.69  1.22  1.892  65.69  65.70  64.47  64.00  66.90   
4834  sz301312   智立方  89.80  4.88  5.747  89.80  89.81  84.92  84.02  92.00   

         最低        成交量          成交额  
0     10.01  

In [3]:
# 0. 准备工作

dbname = "k_house"
#   0.1 创建数据库连接

#  创建数据库 股票分析数据库 的连接 
# conn, curs, engine = common_db.creat_conn_with_pg(dbname=f'{dbname}_for_test')

#  创建数据库 k_house 的连接 
conn2, curs2, engine2 = common_db.creat_conn_with_pg(dbname=dbname)

In [4]:
table_name = 'stock_zh_a_spot_test'

stock_zh_a_spot_df.to_sql(table_name,
          engine2,
          index     = False,
          if_exists = 'replace')    # 写入数据库  还有一个参数是  append，那是附加在表后
# 添加索引
# curs2.execute("""CREATE  INDEX parallel_tool_1      ON  parallel_tool_并行任务分配信息表  (交易日期); 
#                 CREATE  INDEX parallel_tool_2      ON  parallel_tool_并行任务分配信息表  (分组序号);
#                 CREATE  INDEX parallel_tool_3      ON  parallel_tool_并行任务分配信息表  (处理状态);"""
#             )
# conn2.commit()

In [5]:
df = pd.read_sql_query(f"""
    SELECT *
    FROM  {table_name}
  """,
   # params = (date,),
  con = engine2,
)

df[:-5]

,代码,名称,最新价,涨跌额,涨跌幅,买入,卖出,昨收,今开,最高,最低,成交量,成交额
0,bj430047,诺思兰德,10.06,-0.10,-0.984,10.05,10.06,10.16,10.29,10.29,10.01,627178.0,6333588.0
1,bj430090,同辉信息,3.50,-0.03,-0.850,3.50,3.53,3.53,3.58,3.58,3.49,757085.0,2661356.0
2,bj430198,微创光电,5.32,0.00,0.000,5.31,5.32,5.32,5.30,5.32,5.29,80315.0,425749.0
3,bj430418,苏轴股份,8.35,0.01,0.120,8.33,8.35,8.34,8.35,8.45,8.28,51375.0,426912.0
4,bj430489,佳先股份,6.08,-0.02,-0.328,6.03,6.08,6.10,6.07,6.09,6.02,72542.0,439109.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4825,sz301263,泰恩康,28.06,-0.79,-2.738,28.05,28.06,28.85,28.78,29.18,27.80,3425465.0,97030401.0
4826,sz301266,宇邦新材,68.06,0.69,1.024,68.06,68.07,67.37,67.12,70.59,66.53,2254581.0,154395980.0
4827,sz301268,铭利达,55.12,-0.53,-0.952,55.12,55.14,55.65,55.13,55.53,52.60,5291360.0,285152826.0
4828,sz301279,金道科技,26.07,-0.07,-0.268,26.07,26.08,26.14,26.06,26.49,25.85,1678182.0,43884352.0
